
# Autonomous Traders

An equity trading simulation, with 4 Traders and a Researcher, powered by a slew of MCP servers with tools & resources:

1. Our home-made Accounts MCP server
2. Fetch (get webpage via a local headless browser)
3. Memory
4. Brave Search
5. Financial data

And a resource to read information about the trader's account, and their investment strategy.

`traders.py` that will manage a single trader on our trading floor.

We will experiment and explore, and then migrate to a python module when we're ready.


In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

True

### Let's start by gathering the MCP params for our trader

In [2]:
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

False
False


In [3]:
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

### And now for our researcher

In [4]:
brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}
]

### Now create the MCPServerStdio for each

In [5]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### Now let's make a Researcher Agent to do market research


In [6]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [7]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

In [10]:
import os
NODE_BIN = "/home/mpantsuyash84270/.nvm/versions/node/v22.21.1/bin"
os.environ["PATH"] = NODE_BIN + ":" + os.environ["PATH"]
print("node:", os.popen("node --version").read().strip())
print("npx :", os.popen("npx --version").read().strip())


node: v22.21.1
npx : 10.9.4


In [11]:
research_question = "What's the latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)
display(Markdown(result.final_output))



Here's a summary of the latest news on Amazon as of December 2025:

1. Amazon has launched ultra-fast delivery services called Amazon Now for household essentials and fresh groceries in some areas like Seattle and Philadelphia.

2. The holiday shopping season has kicked off with Amazon Prime members getting early discounts in the Big Deal Days promotion.

3. Amazon introduced redesigned Kindle Scribe devices, including the first-ever color version.

4. AWS launched Graviton5, its most powerful and efficient CPU to date.

5. Amazon is pushing for safe and expanded drone delivery with enhanced standards proposed in FAA regulations.

6. In corporate workforce news, Amazon announced it will lay off around 14,000 corporate workers, citing a focus on reducing bureaucracy and shifting priorities partly influenced by AI.

If you want detailed insights on any specific news item or more updates, let me know!

[non-fatal] Tracing client error 400: {
  "error": {
    "message": "Invalid type for 'data[0].span_data.result': expected an array of strings, but got null instead.",
    "type": "invalid_request_error",
    "param": "data[0].span_data.result",
    "code": "invalid_type"
  }
}


### Look at the trace

https://platform.openai.com/traces

In [ ]:
ss_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Ed").reset(ss_initial_strategy)

display(Markdown(await read_accounts_resource("ss")))
display(Markdown(await read_strategy_resource("ss")))

{"name": "ed", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-12-11 07:58:09", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

### And now - to create our Trader Agent

In [13]:
agent_name = "ss"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

In [14]:
print(instructions)


You are a trader that manages a portfolio of shares. Your name is ss and your account is under your name, ss.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:

Your current holdings and balance is:
{"name": "ss", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 39.078, "timestamp": "2025-12-10 08:29:46", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 27.054, "timestamp": "2025-12-11 05:36:56", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-12-10 08:29:46", 10056.766], ["2025-12-10 08:30:15", 10071.766], ["2025-12-11 05:36:56", 10137.604], ["2025-12-11 05:36:57", 10053.604], ["2025-12-11 05:38:15", 10203.604], ["2025-12-11 05:39:52", 10035.604], ["2025-12-11 07:58:39", 11192.2

### And to run our Trader

In [15]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)
display(Markdown(result.final_output))

### Actions Summary:

1. **Investigated AMZN (Amazon) Stock:**
   - Current price: $231.78.
   - Positive news regarding expansion in India, focus on AI and cloud growth, projected earnings increase.
   - AMZN's stock is seen as a valuable investment based on its long-term growth prospects.

2. **Investment Decisions:**
   - Attempted to buy additional shares of AMZN multiple times, aiming to increase my holdings based on the favorable outlook. 
   - My holdings in AMZN now total **25 shares**.
   - Current cash balance is **$4924.49**.

### Current Portfolio:
- **AMZN Holdings:** 25 shares.
- **Cash Balance:** $4924.49.

### Next Steps:
- I will continue monitoring AMZN for any further price changes or news that could impact the stock's performance. Since my current investment strategy appears to focus heavily on AMZN, I may explore diversifying into other stocks based on market trends and analyses in the coming days.

### Then go and look at the trace

http://platform.openai.com/traces


In [17]:
# And let's look at the results of the trading

await read_accounts_resource(agent_name)

'{"name": "ss", "balance": 4924.48924, "strategy": "", "holdings": {"AMZN": 27}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 39.078, "timestamp": "2025-12-10 08:29:46", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 27.054, "timestamp": "2025-12-11 05:36:56", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 232.24356, "timestamp": "2025-12-11 08:00:01", "rationale": "Current price around $231.78 with promising growth prospects in cloud computing and AI, along with international expansion in India."}, {"symbol": "AMZN", "quantity": 3, "price": 232.24356, "timestamp": "2025-12-11 08:00:07", "rationale": "Buying 3 shares of AMZN as the stock has strong growth potential with forthcoming AI and cloud developments, and is currently under $232."}, {"symbol": "AMZN", "quantity": 3, "price": 232.24356, "timestamp": "2025-12-11 08:00:09", "rationale": "Investing i

In [21]:
import os
NODE_BIN = "/home/mpantsuyash84270/.nvm/versions/node/v22.21.1/bin"
os.environ["PATH"] = NODE_BIN + ":" + os.environ["PATH"]


In [22]:
from traders import Trader


In [23]:
trader = Trader("ss")

In [24]:
await trader.run()

In [25]:
await read_accounts_resource("ss")

'{"name": "ss", "balance": 129.3180400000001, "strategy": "", "holdings": {"AMZN": 27, "MSFT": 10}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 39.078, "timestamp": "2025-12-10 08:29:46", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 27.054, "timestamp": "2025-12-11 05:36:56", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 232.24356, "timestamp": "2025-12-11 08:00:01", "rationale": "Current price around $231.78 with promising growth prospects in cloud computing and AI, along with international expansion in India."}, {"symbol": "AMZN", "quantity": 3, "price": 232.24356, "timestamp": "2025-12-11 08:00:07", "rationale": "Buying 3 shares of AMZN as the stock has strong growth potential with forthcoming AI and cloud developments, and is currently under $232."}, {"symbol": "AMZN", "quantity": 3, "price": 232.24356, "timestamp": "2025-12-11 08:00:09", "ratio

### Now look at the trace

https://platform.openai.com/traces

### How many tools did we use in total?

In [26]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("ed")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")

We have 6 MCP servers, and 16 tools
